GET A MPS WITH NATIVE BOND DIMENSION 2

In [1]:
import numpy as np
from ncon import ncon
from numpy import linalg as LA 
from scipy.linalg import null_space as NS 

In [9]:
N = 10
chi = 4
d = 2
mps = np.random.randn((2**10))
mps_norm = mps/LA.norm(mps)


31.62190163337136
0.9999999999999999


In [5]:
mps_sites = [0 for _ in range(N)]
U,S,V = LA.svd(mps_norm.reshape(d,d**(N-1)), full_matrices=False)
mps_sites[0] = U.reshape(d,d)
print(S.shape)
print(V.shape)
print((np.diag(S)@V).shape)

U, S, V = LA.svd((np.diag(S)@V).reshape(d**2,-1), full_matrices=False)
mps_sites[1] = U.reshape(d,d,d**2)
U, S, V = LA.svd((np.diag(S)@V).reshape(d**3,-1), full_matrices=False)
mps_sites[2] = U.reshape(d**2,d,d**3)

onb_3 = ncon([mps_sites[0], mps_sites[1], mps_sites[2]],[[-1,1],[1,-2,2],[2,-3,-4]])
print(onb_3.reshape(d**3,d**3))
print(onb_3.reshape(d**3, d**3)[1]@onb_3.reshape(d**3, d**3)[2].T)

(2,)
(2, 512)
(2, 512)
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]
0.0


In [6]:
mps_sites = [0 for _ in range(N)]


Atemp, Stemp, Vhtemp = LA.svd(mps_GHZ.reshape(2,2**(N-1)), full_matrices=False)
mps_sites[0] = Atemp.reshape(1,2,2).transpose(0,2,1)  #shape della matrice A_1 ---> A_8 paper è 2,2 - mapping 1 a 1
mps_sites[1] = (np.diag(Stemp)@Vhtemp)/LA.norm(Stemp)

for i in range(1,N-1):
    Atemp, Stemp, Vhtemp = LA.svd(mps_sites[i].reshape(2**2,2**(N-i-1)), full_matrices=False)
    mps_sites[i] = Atemp[:,:2].reshape(2,2,2).transpose(0,2,1)
    mps_sites[i+1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,-1))/LA.norm(Stemp)

mps_sites[-1] = ((np.diag(Stemp[:2])@Vhtemp[:2,:]).reshape(2,2,1)).transpose(0,2,1)/LA.norm(Stemp)

mps_2 = ncon([mps_sites[0], mps_sites[1], mps_sites[2]],[[-1,1,-2],[1,2,-3],[2,-4,-5]])
print("Error from MPS to MPS(2):\n",LA.norm(mps_GHZ.reshape(2,2,2) - mps_2[0,:,:,0,:]) / LA.norm(mps_GHZ))

#ora l'ordinamento degli indici è A[j,k,l] con j,k indici virtuali e l indice fisico
#nella contrazione quindi si seguirà lo schema A_jkl A_jkl' = A_ll' con l,l' indici fisici corrispondenti al 1° e 2° sito


ValueError: cannot reshape array of size 1024 into shape (2,2,2)

In [35]:


print(mps_sites[0].shape)
k_a1 = NS(mps_sites[0].transpose(0,2,1).reshape(2,2).conj().T)
print(mps_sites[1].shape)
k_a2 = NS(mps_sites[1].transpose(0,2,1).reshape(4,2).conj().T)
print(mps_sites[2].shape)
k_a3 = NS(mps_sites[2].transpose(0,2,1).reshape(4,1).conj().T)
print(k_a3)

(1, 2, 2)
(2, 2, 2)
(2, 1, 2)
[[0.         0.         0.70710678]
 [1.         0.         0.        ]
 [0.         1.         0.        ]
 [0.         0.         0.70710678]]


In [31]:
for k in null_spaces:
    print("----")
    print("Null space:\n",k)
    print("Shape:\n",k.shape)
    print("Identity:\n",k.conj().T@k)

----
Null space:
 []
Shape:
 (2, 0)
Identity:
 []
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]
----
Null space:
 [[0. 0.]
 [0. 1.]
 [1. 0.]
 [0. 0.]]
Shape:
 (4, 2)
Identity:
 [[1. 0.]
 [0. 1.]]


In [18]:
import numpy as np
from ncon import ncon
from numpy import linalg as LA 
from scipy.linalg import null_space as NS 


In [36]:
Nsites = 10
chid = 2
chi = 5

In [37]:

A = [0 for x in range(Nsites)]
A[0] = np.random.rand(1,chid,min(chi,chid))
A[0] = A[0]/LA.norm(A[0])
for k in range(1,Nsites):
    A[k] = np.random.rand(A[k-1].shape[2],chid,min(min(chi,A[k-1].shape[2]*chid),chid**(Nsites-k-1)))
    A[k] = A[k]/LA.norm(A[k])

In [38]:
for i in A:
    print(i.shape)

(1, 2, 2)
(2, 2, 4)
(4, 2, 5)
(5, 2, 5)
(5, 2, 5)
(5, 2, 5)
(5, 2, 5)
(5, 2, 4)
(4, 2, 2)
(2, 2, 1)


In [39]:
A_tot = ncon([A[0][0,:,:],A[1],A[2],A[3],A[4][:,:,0]],[[-1,1],[1,-2,2],[2,-3,3],[3,-4,4],[4,-5]])

In [40]:
A_tot = A_tot/LA.norm(A_tot)

In [41]:
print(A_tot)

[[[[[0.26820228 0.15716297]
    [0.26462995 0.19799846]]

   [[0.22491561 0.12908514]
    [0.22395491 0.16015916]]]


  [[[0.32945799 0.1917857 ]
    [0.32444568 0.24147018]]

   [[0.26430513 0.15379643]
    [0.26524291 0.19342295]]]]



 [[[[0.12064239 0.07114875]
    [0.11962603 0.08967055]]

   [[0.09655491 0.05562918]
    [0.09654181 0.06963618]]]


  [[[0.12552874 0.07264814]
    [0.1232681  0.09161725]]

   [[0.09815622 0.05699364]
    [0.09836211 0.0718842 ]]]]]
